### Corona Tweets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece 

     |████████████████████████████████| 1.2 MB 10.8 MB/s 


In [3]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 12.4 MB/s 
     |████████████████████████████████| 895 kB 19.4 MB/s 
     |████████████████████████████████| 636 kB 37.8 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [5]:
!unzip -qq /content/drive/MyDrive/datasets/coronaTweet.zip

In [8]:
df = pd.read_csv('/content/Corona_NLP_train.csv', encoding='latin-1')

In [9]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cuda


In [10]:
df_red = df[['OriginalTweet', 'Sentiment']].iloc[:1000]

In [11]:
df_red['sen_enc'] = LabelEncoder().fit_transform(df_red['Sentiment'])

In [12]:
df_red.head()

,OriginalTweet,Sentiment,sen_enc
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,3
1,advice Talk to your neighbours family to excha...,Positive,4
2,Coronavirus Australia: Woolworths to give elde...,Positive,4
3,My food stock is not the only one which is emp...,Positive,4
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,0


## Загружаем модель
Список предобученных моделей: [https://huggingface.co/transformers/pretrained_models.html](https://huggingface.co/transformers/pretrained_models.html)

In [13]:
# DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, 
                                                    ppb.DistilBertTokenizer, 
                                                    'distilbert-base-uncased')

# AlBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.AlbertModel, 
#                                                     ppb.AlbertTokenizerFast,
#                                                     'albert-base-v1')

# BERT
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, 
#                                                     ppb.BertTokenizer, 
#                                                     'bert-base-uncased')

# Load pretrained model/tokenizer

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
tokenized = df_red['OriginalTweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, 
                                                                      truncation=True, 
                                                                      max_length=128)))

In [15]:
len(tokenized[3])

90

In [16]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
max_len

128

#### Masking
Обозначим, где мы добавили маску, чтобы модель не учитывала эту область при обработке

In [18]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 128)

In [19]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)


# Получаем выход модели
with torch.inference_mode():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [20]:
features = last_hidden_states[0][:,0,:].numpy()

In [21]:
features.shape

(1000, 768)

#### используем выход BERT как выборку для обучения классификатора

In [22]:
train_features, test_features, train_labels, test_labels = train_test_split(features, df_red['sen_enc'])

In [23]:
lr_clf = GradientBoostingClassifier(learning_rate=0.001)
lr_clf.fit(train_features, train_labels)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.001, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [24]:
lr_clf.score(test_features, test_labels)

0.232

### IMDB

In [9]:
!pip install sentencepiece 

     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [10]:
!pip install transformers

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import STOPWORDS # more stopwords in gensim corpus
 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
import re

from sklearn.manifold import TSNE

stop_nltk = stopwords.words('english') # nltk corpus

STOPWORDS = STOPWORDS.union(set(["don't", "i'm"]))

In [14]:
reg_tok = RegexpTokenizer('\w+')

In [15]:
def clean_and_lemmatize(text):
    #cleaning
    text = text.lower()
    text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'@\w+',' ',text)
    text = re.sub(r'#\w+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('r<.*?>',' ', text)
    text = reg_tok.tokenize(text)
    
    # filtering
    text = " ".join([word for word in text if not word in STOPWORDS and len(word) > 2])
    
    # lemmatization
    text = WordNetLemmatizer().lemmatize(text)
    text = text.split()
    
    
    return text

In [22]:
!unzip -q //content/drive/MyDrive/datasets/IMDBDataset.csv.zip
!ls

 drive	'IMDB Dataset.csv'   sample_data


In [23]:
df = pd.read_csv('/content/IMDB Dataset.csv', encoding='latin-1')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cuda


In [24]:
df_red = df[['review', 'sentiment']].iloc[:1000]

In [25]:
df_red['sen_enc'] = LabelEncoder().fit_transform(df_red['sentiment'])

In [ ]:
import nltk
nltk.download('wordnet')

In [26]:
df_red['review'] = df_red['review'].apply(clean_and_lemmatize)
df_red.head(15)

LookupError: ignored

In [27]:
# DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, 
#                                                    ppb.DistilBertTokenizer, 
#                                                    'distilbert-base-uncased')

# AlBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.AlbertModel, 
#                                                     ppb.AlbertTokenizerFast,
#                                                     'albert-base-v1')

# BERT
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, 
                                                     ppb.BertTokenizer, 
                                                     'bert-base-uncased')

# Load pretrained model/tokenizer

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
tokenized = df_red['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, 
                                                                      truncation=True, 
                                                                      max_length=128)))

In [ ]:
len(tokenized[3])

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
max_len

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)


# Получаем выход модели
with torch.inference_mode():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features.shape

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, df_red['sen_enc'])

In [ ]:
lr_clf = GradientBoostingClassifier(learning_rate=0.001)
lr_clf.fit(train_features, train_labels)

In [ ]:
lr_clf.score(test_features, test_labels)

Обработка+лемматизация+стоп-слова+ distilbert =0.604

